# Programming Assignment: Рекомендательные системы

## Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок «Смотрели ранее» — в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корелляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать `recall@k` и `precision@k`.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке — например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

## Данные

Вам дается две выборки с пользовательскими сессиями – `id`-шниками просмотренных и `id`-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: `id` просмотренных товаров через `,` затем идёт `;`, после чего следуют `id` купленных товаров (если такие имеются), разделённые запятой. Например, `1,2,3,4;` или `1,2,3,4;5,6`.

Гарантируется, что среди `id` купленных товаров все различные.

**Важно:**
* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и `k` в `recall@k`/`precision@k`.


## Задание

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [36]:
def str2list(string):
    if len(string) == 0:
        return None
    try: 
        return np.array(string.split(','),dtype=np.uint32)
    except ValueError as e:
        print('string: ', string)

### Задание 1. 

На обучении постройте частоты появления `id` в просмотренных и в купленных (`id` может несколько раз появляться в просмотренных, все появления надо учитывать).

In [37]:
data_train = pd.read_table("coursera_sessions_train.txt",sep=';',header=None,converters={0:str2list, 1:str2list})
data_test = pd.read_table("coursera_sessions_train.txt",sep=';',header=None,converters={0:str2list, 1:str2list})

data_train.columns=['see','buy']
data_test.columns=['see','buy']

In [42]:
data_train.head()

,see,buy
0,"[0, 1, 2, 3, 4, 5]",None
1,"[9, 10, 11, 9, 11, 12, 9, 11]",None
2,"[16, 17, 18, 19, 20, 21]",None
3,"[24, 25, 26, 27, 24]",None
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",None


In [51]:
train_see = np.concatenate(data_train.see.values)
train_see_vc = pd.Series(train_see).value_counts()
ts_index = train_see_vc.index.values

In [95]:
train_buy = np.concatenate(data_train.buy.dropna().values)

### Задание 2. 

Реализуйте два алгоритма рекомендаций: сортировка просмотренных `id` по популярности (частота появления в просмотренных), сортировка просморенных `id` по покупаемости (частота появления в покупках). Если частота одинаковая, то сортировать надо по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [61]:
np.intersect1d(ts_index[:5],train_buy)

array([ 73, 158, 162, 204, 262])

In [57]:
ts_index[:5]

array([ 73, 158, 204, 262, 162])

In [85]:
def Precision(see, buy , k):
    return np.intersect1d(see[:k], buy).shape[0]/float(5)

In [82]:
def Recall(see, buy, k):
    return np.intersect1d(see[:k],buy).shape[0]/float(len(buy))

### Задание 3. 

Для данных алгоритмов выпишите через пробел `AverageRecall@1`, `AveragePrecision@1`, `AverageRecall@5`, `AveragePrecision@5` на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

### Дополнительные вопросы

* Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров `recall@k` меняется. Подумайте, как оценить минимальное и максимальное значение `recall@k` в зависимости от правила сортировки.
* Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.